In [40]:
# The file generated by this code is currently used to prepare the “Rider Behavior Overview.”

import os, pandas as pd, re, numpy as np
from tqdm import tqdm

BASE_DIR = "/Users/saurabh/Documents/New Haven/Curriculam/GA/Citi Bikes/datasets/citibike_2025_raw"
COL_ORDER = ["ride_id","rideable_type","started_at","ended_at","start_station_name","start_station_id","end_station_name","end_station_id","start_lat","start_lng","end_lat","end_lng","member_casual"]

all_csv_paths = [os.path.join(root,f) for root,dirs,files in os.walk(BASE_DIR) for f in files if f.endswith('.csv')]
print(f"Found {len(all_csv_paths)} files")

all_dfs = []
for full_path in tqdm(all_csv_paths):
    filename = os.path.basename(full_path)
    df = pd.read_csv(full_path, low_memory=False)
    df = df.reindex(columns=COL_ORDER)
    
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
    df['month_year'] = df['started_at'].dt.strftime('%Y-%m')
    df['month_name'] = df['started_at'].dt.strftime('%B %Y')
    df['year'] = df['started_at'].dt.year
    df['month'] = df['started_at'].dt.month
    
    all_dfs.append(df)

# Combine + analyze
combined = pd.concat(all_dfs, ignore_index=True)
combined = combined.dropna(subset=['started_at','ended_at','start_station_name','end_station_name'])
combined['hour_started_at'] = combined['started_at'].dt.hour
combined['ride_duration_sec'] = (combined['ended_at']-combined['started_at']).dt.total_seconds()

combined = combined[combined['ride_duration_sec']>0]

# Calculate distance
with np.errstate(invalid='ignore'):
    combined['distance_miles'] = 3959 * np.arccos(
        np.sin(np.radians(combined['start_lat'])) * np.sin(np.radians(combined['end_lat'])) +
        np.cos(np.radians(combined['start_lat'])) * np.cos(np.radians(combined['end_lat'])) * 
        np.cos(np.radians(combined['start_lng']) - np.radians(combined['end_lng']))
    )
    

daily_hourly = combined.groupby(['month_year','month_name','hour_started_at','start_station_name','member_casual']).agg({
    'ride_id':'count',
    'ride_duration_sec':['sum','mean','median'],
    'distance_miles': 'sum'
}).reset_index().droplevel(axis=1,level=0)

daily_hourly.columns = ['month_year','month_name','hour_started_at','start_station_name','member_casual',
                       'trips','total_ride_duration_sec','avg_ride_duration_sec','median_ride_duration_sec',
                       'total_distance_miles']

for col in ['avg_ride_duration_min','median_ride_duration_min','total_ride_duration_min']:
    daily_hourly[col] = daily_hourly[col.replace('_min','_sec')] / 60


# Save
combined.to_csv("citibike_2025_cleaned_full.csv", index=False)
daily_hourly.to_csv("citibike_2025_daily_hourly_stations.csv", index=False)

print(f"DONE! FULL: {combined.shape} | DAILY: {daily_hourly.shape}")


Found 50 files


100%|███████████████████████████████████████████| 50/50 [05:19<00:00,  6.38s/it]


DONE! FULL: (43545978, 20) | DAILY: (997883, 13)


In [28]:
# This code is used to validate aggregated data by performing random checks for selected months, hours, and stations.

import os
import pandas as pd
import re
from tqdm import tqdm

# CONFIG - CHANGE HERE ONLY
BASE_DIR = "/Users/saurabh/Documents/New Haven/Curriculam/GA/Citi Bikes/datasets/citibike_2025_raw"
MONTH_TARGET = "202501"  # January 2025
HOUR_FILTER = [12]  # or [7,8,9]
STATION_FILTER = ["S 2 St & Kent Ave"]  # or ["W 20 St & 7 Ave"]

print(f"Merging {MONTH_TARGET} files...")

# Find MONTH files only
month_files = []
for root, dirs, files in os.walk(BASE_DIR):
    for file in files:
        if file.lower().endswith('.csv') and re.search(rf'{MONTH_TARGET}', file):
            month_files.append(os.path.join(root, file))

print(f"Found {len(month_files)} {MONTH_TARGET} files")

# Merge ALL month files
all_dfs = []
for file_path in tqdm(month_files, desc="Merging"):
    df = pd.read_csv(file_path)
    all_dfs.append(df)

print("Merging dataframes...")
merged = pd.concat(all_dfs, ignore_index=True)
print(f"Raw merged: {len(merged):,} rows")

# Convert datetime ONLY (no other changes)
merged['started_at'] = pd.to_datetime(merged['started_at'], errors='coerce')
merged['ended_at'] = pd.to_datetime(merged['ended_at'], errors='coerce')

# Add hour column ONLY
merged['hour_started_at'] = merged['started_at'].dt.hour

# Filter (optional)
if HOUR_FILTER:
    merged = merged[merged['hour_started_at'].isin(HOUR_FILTER)]
if STATION_FILTER:
    merged = merged[merged['start_station_name'].isin(STATION_FILTER)]

print(f"After filters: {len(merged):,} rows")
print(f"Hours range: {merged['hour_started_at'].min()}-{merged['hour_started_at'].max()}")

# Save raw merged data
merged.to_csv(f"{MONTH_TARGET}_merged_filtered.csv", index=False)
print(f"Saved: {MONTH_TARGET}_merged_filtered.csv")

# Quick check
print("\nHour distribution:")
print(merged['hour_started_at'].value_counts().sort_index().head(10))
print("\nSample data:")
print(merged[['started_at', 'ended_at', 'hour_started_at', 'start_station_name']].head())


Merging 202501 files...
Found 3 202501 files


Merging:   0%|                                            | 0/3 [00:00<?, ?it/s]/var/folders/rp/mkwcs2k94r570kpc9y6hsywc0000gn/T/ipykernel_35595/1584646144.py:27: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Merging:  67%|████████████████████████            | 2/3 [00:02<00:00,  1.13it/s]/var/folders/rp/mkwcs2k94r570kpc9y6hsywc0000gn/T/ipykernel_35595/1584646144.py:27: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Merging: 100%|████████████████████████████████████| 3/3 [00:03<00:00,  1.30s/it]


Merging dataframes...
Raw merged: 2,124,475 rows
After filters: 180 rows
Hours range: 12-12
✅ Saved: 202501_merged_filtered.csv

Hour distribution:
hour_started_at
12    180
Name: count, dtype: int64

Sample data:
                   started_at                ended_at  hour_started_at  \
268   2025-01-07 12:42:07.719 2025-01-07 12:44:55.044               12   
6661  2025-01-12 12:36:41.926 2025-01-12 12:59:55.533               12   
11827 2025-01-09 12:55:29.886 2025-01-09 13:03:36.787               12   
24017 2025-01-08 12:25:36.259 2025-01-08 12:41:43.534               12   
25222 2025-01-10 12:26:45.532 2025-01-10 12:29:04.530               12   

      start_station_name  
268    S 2 St & Kent Ave  
6661   S 2 St & Kent Ave  
11827  S 2 St & Kent Ave  
24017  S 2 St & Kent Ave  
25222  S 2 St & Kent Ave  


In [46]:
# This code creates a separate file for the station-level dashboard. It is currently not in use because the number of rows generated exceeds the BI tool’s capacity.

import os, pandas as pd, numpy as np
from tqdm import tqdm

BASE_DIR = "/Users/saurabh/Documents/New Haven/Curriculam/GA/Citi Bikes/datasets/citibike_2025_raw"
COL_ORDER = ["ride_id","rideable_type","started_at","ended_at","start_station_name","start_station_id","end_station_name","end_station_id","start_lat","start_lng","end_lat","end_lng","member_casual"]

# Load + clean
all_csv_paths = [os.path.join(root,f) for root,dirs,files in os.walk(BASE_DIR) for f in files if f.endswith('.csv')]
print(f"Found {len(all_csv_paths)} files")

all_dfs = []
for full_path in tqdm(all_csv_paths, desc="Loading"):
    df = pd.read_csv(full_path, low_memory=False)
    df = df.reindex(columns=COL_ORDER)
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
    df['month_year'] = df['started_at'].dt.strftime('%Y-%m')   
    df['hour_started_at'] = df['started_at'].dt.hour    
    all_dfs.append(df)

combined = pd.concat(all_dfs, ignore_index=True)
combined = combined.dropna(subset=['started_at','ended_at','start_station_name','end_station_name'])
combined['hour_started_at'] = combined['started_at'].dt.hour
combined['ride_duration_sec'] = (combined['ended_at']-combined['started_at']).dt.total_seconds()
combined = combined[combined['ride_duration_sec']>0]

# Distance
print("Calculating distances...")
with np.errstate(invalid='ignore'):
    combined['distance_miles'] = 3959 * np.arccos(
        np.sin(np.radians(combined['start_lat'])) * np.sin(np.radians(combined['end_lat'])) +
        np.cos(np.radians(combined['start_lat'])) * np.cos(np.radians(combined['end_lat'])) * 
        np.cos(np.radians(combined['start_lng']) - np.radians(combined['end_lng']))
    )

print(f"Raw rides: {combined.shape[0]:,}")


print("Creating hourly station flows...")
station_flows = combined.groupby(['month_year','hour_started_at','start_station_name','end_station_name','member_casual']).agg({
    'ride_id':'count',           # total_trips
    'ride_duration_sec':'sum',   # total_trip_duration_sec
    'distance_miles':'sum'       # total_distance_miles
}).reset_index()

station_flows.columns = ['month_year','hour_started_at','start_station_name','end_station_name','member_casual',
                        'total_trips','total_trip_duration_sec','total_distance_miles']

# Convert to minutes
station_flows['total_trip_duration_min'] = station_flows['total_trip_duration_sec'] / 60


station_flows = station_flows[['month_year', 'hour_started_at', 'start_station_name', 'end_station_name', 
                              'member_casual', 'total_trips', 'total_trip_duration_min', 
                              'total_distance_miles']]


# Save
output_file = "citibike_2025_station_flows_hourly.csv"
station_flows.to_csv(output_file, index=False)

print(f"\nSAVED: {output_file}")
print(f"Columns: {list(station_flows.columns)}")


Found 50 files


Loading: 100%|██████████████████████████████████| 50/50 [14:00<00:00, 16.80s/it]


Calculating distances...
Raw rides: 43,545,978
Creating hourly station flows...

SAVED: citibike_2025_station_flows_hourly.csv
Columns: ['month_year', 'hour_started_at', 'start_station_name', 'end_station_name', 'member_casual', 'total_trips', 'total_trip_duration_min', 'total_distance_miles']


In [48]:
# This improved code includes all required columns for both Rider and Station dashboards. 
# It is the preferred approach but is not currently used due to limitations of the free BI tool.

import os, pandas as pd, numpy as np
from tqdm import tqdm


BASE_DIR = "/Users/saurabh/Documents/New Haven/Curriculam/GA/Citi Bikes/datasets/citibike_2025_raw"
COL_ORDER = ["ride_id","rideable_type","started_at","ended_at","start_station_name","start_station_id","end_station_name","end_station_id","start_lat","start_lng","end_lat","end_lng","member_casual"]

# Load + clean
all_csv_paths = [os.path.join(root,f) for root,dirs,files in os.walk(BASE_DIR) for f in files if f.endswith('.csv')]
print(f"Found {len(all_csv_paths)} files")

all_dfs = []
for full_path in tqdm(all_csv_paths, desc="Loading"):
    df = pd.read_csv(full_path, low_memory=False)
    df = df.reindex(columns=COL_ORDER)
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
    df['month_year'] = df['started_at'].dt.strftime('%Y-%m')
    df['month_name'] = df['started_at'].dt.strftime('%B %Y')
    df['year'] = df['started_at'].dt.year
    df['month'] = df['started_at'].dt.month
    all_dfs.append(df)

# Combine + filter
combined = pd.concat(all_dfs, ignore_index=True)
combined = combined.dropna(subset=['started_at','ended_at','start_station_name','end_station_name'])
combined['hour_started_at'] = combined['started_at'].dt.hour
combined['ride_duration_sec'] = (combined['ended_at']-combined['started_at']).dt.total_seconds()
combined = combined[combined['ride_duration_sec']>0]

# Distance
print("Calculating distances...")
with np.errstate(invalid='ignore'):
    combined['distance_miles'] = 3959 * np.arccos(
        np.sin(np.radians(combined['start_lat'])) * np.sin(np.radians(combined['end_lat'])) +
        np.cos(np.radians(combined['start_lat'])) * np.cos(np.radians(combined['end_lat'])) * 
        np.cos(np.radians(combined['start_lng']) - np.radians(combined['end_lng']))
    )


print("Creating SLIM station flows...")
daily_hourly = combined.groupby(['month_year','hour_started_at','start_station_name','end_station_name','member_casual']).agg({
    'ride_id':'count',           # trips
    'ride_duration_sec':'sum',   # total_duration_sec
    'distance_miles':'sum'       # total_distance_miles
}).reset_index()

daily_hourly.columns = ['month_year','hour_started_at','start_station_name','end_station_name','member_casual',
                       'trips','total_ride_duration_sec','total_distance_miles']

# Convert to minutes (Tableau-friendly)
daily_hourly['total_ride_duration_min'] = daily_hourly['total_ride_duration_sec'] / 60


# Save both
combined.to_csv("citibike_2025_cleaned_full.csv", index=False)
daily_hourly.to_csv("citibike_2025_daily_hourly_stations.csv", index=False)

print(f"\nDone")

Found 50 files


Loading: 100%|██████████████████████████████████| 50/50 [06:34<00:00,  7.89s/it]


Calculating distances...
Creating SLIM station flows...


In [50]:
# In this code, only the top 100 stations are selected. Since the full dataset generates a large number of rows that are not supported by the BI tool, this reduced dataset is used to prepare the Station Network Overview dashboard.

import os, pandas as pd, numpy as np
from tqdm import tqdm

BASE_DIR = "/Users/saurabh/Documents/New Haven/Curriculam/GA/Citi Bikes/datasets/citibike_2025_raw"
COL_ORDER = ["ride_id","rideable_type","started_at","ended_at","start_station_name","start_station_id","end_station_name","end_station_id","start_lat","start_lng","end_lat","end_lng","member_casual"]

# Load + clean
all_csv_paths = [os.path.join(root,f) for root,dirs,files in os.walk(BASE_DIR) for f in files if f.endswith('.csv')]
print(f"Found {len(all_csv_paths)} files")

all_dfs = []
for full_path in tqdm(all_csv_paths, desc="Loading"):
    df = pd.read_csv(full_path, low_memory=False)
    df = df.reindex(columns=COL_ORDER)
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
    df['month_year'] = df['started_at'].dt.strftime('%Y-%m')   
    df['hour_started_at'] = df['started_at'].dt.hour    
    all_dfs.append(df)

combined = pd.concat(all_dfs, ignore_index=True)
combined = combined.dropna(subset=['started_at','ended_at','start_station_name','end_station_name'])
combined['ride_duration_sec'] = (combined['ended_at']-combined['started_at']).dt.total_seconds()
combined = combined[combined['ride_duration_sec']>0]

print(f"Raw rides: {combined.shape[0]:,}")


print("Finding TOP 100 stations...")
top_start = combined['start_station_name'].value_counts().head(100).index
top_end = combined['end_station_name'].value_counts().head(100).index
top_stations = list(set(top_start).union(set(top_end)))  # BOTH!

print(f"Top stations: {len(top_stations)} unique (100 start + 100 end)")


combined = combined[
    combined['start_station_name'].isin(top_stations) & 
    combined['end_station_name'].isin(top_stations)
]

print(f"Filtered rides: {combined.shape[0]:,} ({100*combined.shape[0]/combined.shape[0]:.0%} of total)")

# Distance (only on filtered data)
print("Calculating distances...")
with np.errstate(invalid='ignore'):
    combined['distance_miles'] = 3959 * np.arccos(
        np.sin(np.radians(combined['start_lat'])) * np.sin(np.radians(combined['end_lat'])) +
        np.cos(np.radians(combined['start_lat'])) * np.cos(np.radians(combined['end_lat'])) * 
        np.cos(np.radians(combined['start_lng']) - np.radians(combined['end_lng']))
    )

print("Creating hourly station flows...")
station_flows = combined.groupby(['month_year','hour_started_at','start_station_name','end_station_name','member_casual']).agg({
    'ride_id':'count',
    'ride_duration_sec':'sum',
    'distance_miles':'sum'
}).reset_index()

station_flows.columns = ['month_year','hour_started_at','start_station_name','end_station_name','member_casual',
                        'total_trips','total_trip_duration_sec','total_distance_miles']

station_flows['total_trip_duration_min'] = station_flows['total_trip_duration_sec'] / 60

station_flows = station_flows[['month_year', 'hour_started_at', 'start_station_name', 'end_station_name', 
                              'member_casual', 'total_trips', 'total_trip_duration_min', 
                              'total_distance_miles']]

# Optimize types
station_flows['hour_started_at'] = station_flows['hour_started_at'].astype('int8')

# Save
output_file = "citibike_2025_station_flows_TOP100.csv"
station_flows.to_csv(output_file, index=False)

print(f"\nDone")

Found 50 files


Loading: 100%|██████████████████████████████████| 50/50 [03:47<00:00,  4.56s/it]


Raw rides: 43,545,978
Finding TOP 100 stations...
Top stations: 102 unique (100 start + 100 end)
Filtered rides: 3,247,071 (10000% of total)
Calculating distances...
Creating hourly station flows...


In [52]:
# This code creates a unique list of station names along with their latitude and longitude. The input file used here is a merged dataset created from all raw/sub files.

import pandas as pd

FULL_PATH = "/Users/saurabh/Documents/New Haven/Curriculam/GA/Citi Bikes/Python Code/citibike_2025_cleaned_full.csv"

# 2. Read ONLY the columns needed for stations
use_cols = [
    "start_station_name", "start_lat", "start_lng",
    "end_station_name",   "end_lat",   "end_lng"
]
df = pd.read_csv(FULL_PATH, usecols=use_cols)

print("Loaded shape:", df.shape)

# 3. Build start-station table
start_stations = df[["start_station_name", "start_lat", "start_lng"]].dropna()
start_stations = start_stations.rename(columns={
    "start_station_name": "station_name",
    "start_lat": "lat",
    "start_lng": "lng"
})

# 4. Build end-station table
end_stations = df[["end_station_name", "end_lat", "end_lng"]].dropna()
end_stations = end_stations.rename(columns={
    "end_station_name": "station_name",
    "end_lat": "lat",
    "end_lng": "lng"
})

# 5. Combine and keep unique stations
stations = pd.concat([start_stations, end_stations], ignore_index=True)

# Optional: round coordinates a bit to help deduplicate tiny float differences
stations["lat"] = stations["lat"].round(6)
stations["lng"] = stations["lng"].round(6)

stations = stations.drop_duplicates(subset=["station_name", "lat", "lng"]).reset_index(drop=True)

print("Unique stations:", stations.shape)

# 6. Save the lookup file
stations.to_csv("citibike_2025_stations_lookup.csv", index=False)
print("Saved citibike_2025_stations_lookup.csv with columns:", list(stations.columns))


Loaded shape: (43545978, 6)
Unique stations: (2352, 3)
Saved citibike_2025_stations_lookup.csv with columns: ['station_name', 'lat', 'lng']
